**Documention**

https://spacy.io/usage/training

https://huggingface.co/datasets/eriktks/conll2003

In [ ]:
#!pip install spacy


how to build ner model ?
1. dataset
2. framework that supports building ner models -> spacy
3. what is the shape of the dataset i need to make it work with spacy
4. build training pipeline
5. evaluate

**Dataset**

In [1]:
TRAIN_DATA = [
    ("Apple Inc. is located in Cupertino", {"entities": [(0, 10, "ORG"), (25, 35, "LOC")]}),
    ("Tim Cook is the CEO of Apple", {"entities": [(0, 8, "PER"), (23, 28, "ORG")]})
]


**Build Training Pipeline**

In [3]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch
import random

# Load a blank English model
nlp = spacy.blank("en")

# Create a new NER pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

# Add labels to the NER
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Disable other pipelines during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for i in range(30):  # 30 training iterations
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=2)
        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.5, losses=losses)
        print(f"Losses at iteration {i}: {losses}")


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Apple Inc. is located in Cupertino" with entities "[(0, 10, 'ORG'), (25, 35, 'LOC')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses at iteration 0: {'ner': np.float32(11.253367)}
Losses at iteration 1: {'ner': np.float32(10.811395)}
Losses at iteration 2: {'ner': np.float32(10.118758)}
Losses at iteration 3: {'ner': np.float32(9.846732)}
Losses at iteration 4: {'ner': np.float32(8.942562)}
Losses at iteration 5: {'ner': np.float32(8.103602)}
Losses at iteration 6: {'ner': np.float32(7.523084)}
Losses at iteration 7: {'ner': np.float32(6.4966416)}
Losses at iteration 8: {'ner': np.float32(5.6662946)}
Losses at iteration 9: {'ner': np.float32(4.757847)}
Losses at iteration 10: {'ner': np.float32(4.3821936)}
Losses at iteration 11: {'ner': np.float32(4.4335003)}
Losses at iteration 12: {'ner': np.float32(3.852095)}
Losses at iteration 13: {'ner': np.float32(4.393154)}
Losses at iteration 14: {'ner': np.float32(4.9026237)}
Losses at iteration 15: {'ner': np.float32(5.1164093)}
Losses at iteration 16: {'ner': np.float32(4.9709697)}
Losses at iteration 17: {'ner': np.float32(4.561981)}
Losses at iteration 18: {'ne

**Evaluate the Model**

In [4]:
test_text = "Apple is opening a new office in Cupertino"
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.label_)


Apple is ORG
